<a href="https://colab.research.google.com/github/ShaelinN/VAE_RECOMMENDER_IMPLICIT_FEEDBACK/blob/main/VAE_V3_primary_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports and Dirs

In [ ]:
#!pip3 install tensorflow-ranking
#!pip3 install tensorflow
#!pip3 install numpy
#!pip3 install pickle
#!pip3 install math

import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as tfback
from tensorflow.keras import layers, activations, Model, losses
from tensorflow.keras.callbacks import Callback, ModelCheckpoint
#import tensorflow_ranking as tfr

import numpy as np
import os
import pickle
import math
import argparse

In [ ]:
"""
#uncomment if running as .py on cluster such as HIPPO
#comment if running as .ipynb on Google Colab
parser = argparse.ArgumentParser()
parser.add_argument('--root', type=str)
parser.add_argument('--input_data', type=str)
parser.add_argument('--training_results', type=str, default="training_results")

parser.add_argument('--prim_pel', type=str, default="prim_pel.txt")
parser.add_argument('--prim_pbl', type=str, default="prim_pbl.txt")
parser.add_argument('--prim_pevl', type=str, default="prim_pevl.txt")

parser.add_argument('--sec_pel', type=str, default="sec_pel.txt")
parser.add_argument('--sec_pbl', type=str, default="sec_pbl.txt")
parser.add_argument('--sec_pevl', type=str, default="sec_pevl.txt")


parser.add_argument('--weights', type=str)
parser.add_argument('--intermediatedim', type=int, default=512)
parser.add_argument('--epochs', type=int, default=5)
parser.add_argument('--batchsize', type=int, default=200)
parser.add_argument('--klannealrate', type=float, default=0.0001)
args = parser.parse_args()
"""

In [ ]:
#uncomment if running as .ipynb on Google Colabas
#comment if running as  .py on cluster such as HIPPO
class argclass(object):
  def __init__(self):
    self.root = "/content/drive/MyDrive/COMP700_Honours Project"
    self.input_data = "Data/remove_low_interaction_users/split/matrices/implicit"
    self.training_results ="training_results"

    self.prim_pel = "prim_pel.txt"
    self.prim_pbl = "prim_pbl.txt"
    self.prim_pevl = "prim_pevl.txt"

    self.sec_pel = "sec_pel.txt"
    self.sec_pbl = "sec_pbl.txt"
    self.sec_pevl = "sec_pevl.txt"

    self.weights = None #"vae_epoch_01_loss_210.97.hdf5"
    self.intermediatedim = 512 
    self.epochs = 5
    self.batchsize = 200
    self.klannealrate = 0.0001
args = argclass()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
root = args.root

input_data = args.input_data
input_data = os.path.join(root, input_data)

training_results = args.training_results
training_results = os.path.join(root, training_results)



try:
  os.mkdir(training_results)
except FileExistsError:
  pass

#IF NO WEIGHTS FILE TO USE, THEN JUST LEAVE ARG OUT. THE LAOD FUNCTION WILL KNOW TO START ANEW
# Checkpoint callback saves into filepath=os.path.join(training_results,"vae_epoch_{epoch:02d}_loss_{loss:.2f}.hdf5")
weights = args.weights
try:
  weights = os.path.join(training_results, weights)
except TypeError:
  pass

#annealing_rate = args.klannealrate
#intermediate_dim = args.intermediatedim
#total_num_epochs = args.epochs
#batch_size = args.batchsize



print(root)
print(input_data)
print(training_results)
print(weights)

/content/drive/MyDrive/COMP700_Honours Project
/content/drive/MyDrive/COMP700_Honours Project/Data/remove_low_interaction_users/split/matrices/implicit
/content/drive/MyDrive/COMP700_Honours Project/training_results
None


#Model design
* sample layer
* batch history callback
* vae loss function
* VAE builder class
* data generator sequence

In [ ]:
class Sampling(layers.Layer):
  def __init__(self, name="Sampling", **kwargs):
    super(Sampling, self).__init__(name=name, **kwargs)
    
  def call(self, inputs):
    z_mean, z_log_var = inputs
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]


    #epsilon = distribution.sample()
    
    epsilon = tfback.random_normal(shape=(batch,dim))
    sample = epsilon * tf.exp(0.5 * z_log_var)  +   z_mean  #Reparametrization trick: convert from standard normal to desired distribution

    return sample

In [ ]:
class BatchHistory(keras.callbacks.Callback):  
  def __init__(self, pel, pbl, pevl):
    super(BatchHistory,self).__init__() 
    self.loss = [] 
    self.val_loss = 0.0
    self.pel = pel
    self.pbl = pbl
    self.pevl = pevl

  def on_train_batch_end(self, batch, logs=None):   
    self.loss.append(logs.get('loss'))


  def on_epoch_end(self, epoch, logs=None):
    self.val_loss = logs.get('val_loss')
    self.loss = [str(i) for i in self.loss]

    with open(os.path.join(training_results,self.pevl), "a") as pe_v_loss:
      pe_v_loss.write('\n'+ str(self.val_loss))

    with open(os.path.join(training_results,self.pbl), "a") as pb_loss:
      pb_loss.write("\n")
      pb_loss.writelines('\n'.join(self.loss))

    with open(os.path.join(training_results,self.pel), "a") as pe_loss:
      pe_loss.write('\n'+ str(self.loss[len(self.loss)-1]))
    self.loss = []
    

In [ ]:
#original_dim = 127350
annealing_rate = args.klannealrate
KLBeta = 0
def VAE_loss(y_true, y_pred):
    global KLBeta
    reconst_loss =  original_dim * losses.binary_crossentropy(y_true, y_pred)
    KLDiv = KLBeta * losses.kl_divergence(y_true, y_pred)

    KLBeta = min(KLBeta+annealing_rate , 1) #update weight of KL factor

    #return (KLDiv,reconst_loss)

    return  reconst_loss + KLDiv


In [ ]:
class vae_builder(object):
  def __init__(self, original_dim, intermediate_dim, latent_dim, name='VAE'):
    self.name = name
    self.original_dim = original_dim
    self.intermediate_dim = intermediate_dim
    self.latent_dim = latent_dim

  
  def build(self):
    self.input = layers.Input(self.original_dim, name = 'input')
    self.dropout = layers.Dropout(rate=0.5)(self.input)

    #encoder
    self.e1 = layers.Dense(self.intermediate_dim, activation=activations.hard_sigmoid, name = 'e1')(self.dropout)
    self.e2 = layers.Dense(self.intermediate_dim, activation=activations.hard_sigmoid, name = 'e2')(self.e1)
    self.e3 = layers.Dense(self.intermediate_dim, activation=activations.hard_sigmoid, name = 'e3')(self.e2)
    self.e4 = layers.Dense(self.intermediate_dim, activation=activations.hard_sigmoid, name = 'e4')(self.e3)
    self.e5 = layers.Dense(self.intermediate_dim, activation=activations.hard_sigmoid, name = 'e5')(self.e4)

    #sampling
    self.mean = layers.Dense(self.latent_dim, name = 'mean')(self.e5)  
    self.log_var = layers.Dense(self.latent_dim, name = 'log_var')(self.e5)  
    self.sampling = Sampling()([self.mean, self.log_var])

    #decoder
    self.d1 = layers.Dense(self.intermediate_dim, activation='relu',name = 'd1')(self.sampling)
    self.d2 = layers.Dense(self.original_dim, activation='relu', name = 'd2')(self.d1)  
    
    self.output = layers.Activation(activations.swish, name = 'output')(self.d2)

    #to model
    vae = Model(inputs = self.input, outputs = self.output, name = self.name)
    return vae

In [ ]:
class datagen(keras.utils.Sequence):
  def __init__(self, x_set, batch_size, max_samples_per_epoch=None):
    self.x = x_set
    self.batch_size = batch_size

    self.shuffled_idx = np.arange(np.shape(self.x)[0]) #get indexes
    np.random.shuffle(self.shuffled_idx) # shuffle

    if max_samples_per_epoch is not None:
      self.shuffled_idx = self.shuffled_idx[:max_samples_per_epoch] #cutoff at max no of samples allowed in epoch

  def on_epoch_end(self):
    self.shuffled_idx = np.arange(np.shape(self.x)[0]) #get indexes
    np.random.shuffle(self.shuffled_idx) # shuffle

  def __len__(self):
      #return math.ceil((self.x.shape[0]) / self.batch_size)
      return len(self.shuffled_idx)
      
  def __getitem__(self, idx):
    b_idx = idx * self.batch_size
    e_idx = (idx + 1) * self.batch_size
    idx = self.shuffled_idx[b_idx:e_idx] #cut slice of indexes using begin and end indexes

    batch_x = np.array(self.x[idx].todense())

    batch_y = np.array(self.x[idx].todense())


    return batch_x , batch_y


#Primary Training

##load data and set params

In [ ]:
train = pickle.load( 
    open(os.path.join(input_data,"train_impl.pkl") , 'rb')
)
vad = pickle.load( 
    open(os.path.join(input_data,"vad_impl.pkl") , 'rb')
)


In [ ]:
total_num_samples = train.shape[0]
original_dim = train.shape[1]
intermediate_dim = args.intermediatedim
latent_dim = intermediate_dim//2 #64

total_num_epochs = args.epochs

batch_size = args.batchsize

In [ ]:
train_gen = datagen(train , batch_size, 10)
vad_gen = datagen(vad ,  batch_size, 10)

##code to allow continuation of training over many sessions

In [ ]:
def load_model(v, weights_file):
  try:
    v.load_weights(weights_file)
    print("weights loaded successfully")
  except:
    print("failed to load weights")

def get_num_epochs_complete():
  try:
    with open(os.path.join(training_results,args.prim_pevl), "r") as f:
      lines = [i for i in f.readlines() if len(i.strip()) > 0]
      return len(lines)
  except FileNotFoundError:
    return 0

##Train code block


In [ ]:
#REFRESH CALLBACKS BEFORE EACH RUN
#or else separate runs in same session will behave oddly
checkpoint = ModelCheckpoint(filepath=os.path.join(training_results,"vae_primary_train_epoch_{epoch:02d}_loss_{loss:.2f}.hdf5"), verbose=1, save_best_only=False)
batch_hist = BatchHistory(args.prim_pel, args.prim_pbl, args.prim_pevl) #use primary train loss logs

#BUILD STRUCTURE
v = vae_builder(original_dim, intermediate_dim, latent_dim).build()

#configurable optimiser
slow_adam = keras.optimizers.Adam(learning_rate=1e-4)


v.compile(optimizer=slow_adam, loss=VAE_loss)

#PICK UP FROM WHERE LEFT OFF
complete_epochs = get_num_epochs_complete()
remaining_epochs = total_num_epochs - complete_epochs
load_model(v, weights)

#RUN
print("commisioned to run for",total_num_epochs,"epochs total")
print("known loss values found for ",complete_epochs,"previous epochs")
print("will run for", remaining_epochs, "more epochs unless stopped early")

v.fit(train_gen, verbose=True, epochs=remaining_epochs, validation_data=vad_gen, callbacks=[checkpoint,batch_hist])

print("complete")